In [ ]:
import os

def delete_videos(dir_path):
    for root, _, files in os.walk(dir_path):
        for file in files:
            if file.endswith(".mp4"):
                os.remove(os.path.join(root, file))

# dir_path = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/formatted_datasets/oprime_result_hidden_nba_36k_train_bal_test_imbal_4s_480"
# delete_videos(dir_path)

In [6]:
from format_data_kinetics_tf import clean_directory, format_dataset_kinetics
from split_dataset import copy_and_split_dataset

dst_dir = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/formatted_datasets/oprime_result_hidden_nba_50k_train_bal_test_bal_4s_480"
test_dir = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/test-sets/result-hidden/nba_.5k_balanced_4s_856x480"

copy_and_split_dataset(
    train_val_dir="/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/raw_clips/nba_results_hidden_30k_480",
    test_dir=test_dir,
    dst_dir=dst_dir,
    num_files_to_copy=50000,
    # class_balance=[0.4529, 0.5471],
)

clean_directory(dst_dir)
format_dataset_kinetics(dst_dir)

 60%|█████▉    | 30217/50500 [02:59<01:37, 207.41it/s]

In [ ]:
import os

fp_made = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/formatted_datasets/oprime_result_hidden_nba_5k_train_bal_test_imbal_4s_480/test/made"
fp_missed = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/formatted_datasets/oprime_result_hidden_nba_5k_train_bal_test_imbal_4s_480/test/missed"
num_made = os.listdir(fp_made).__len__()
num_missed = os.listdir(fp_missed).__len__()
num_made, num_missed

In [ ]:
import pandas as pd

ex_fp = '/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/mixed-leagues/game-logs/Game_log_data/22-23/2_Euroleague/938086_Olimpia Milano - Real Madrid.csv'
df = pd.read_csv(ex_fp, delimiter=';')
df

In [ ]:
import os 

os.listdir(
    "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/formatted_datasets/oprime_result_hidden_nba_5k_train_bal_test_imbal_4s_480/test/made"
)